# Laboratório 5 - Quão eficazes são os ITS implementados com GenAI?

<br>

Os Sistemas de Tutoria Inteligente (ITS) são agentes inteligentes que especificam *o que* ensinar e *como* ensinar de forma adaptativa. Por ser um conceito amplo, comporta diferentes abordagens de implementação.

Neste laboratório, você irá criar uma implementação simplificada de um ITS completo utilizando IA Generativa (GenAI) e refletir sobre a eficácia dessa abordagem. São fornecidos códigos preliminares para apoiar sua jornada.

Ao concluir a implementação, em `📝 Tarefa`, você irá responder à algumas perguntas por meio de testes com dois domínios distintos, refletindo sobre o título deste Laboratório.

> Atividade de laboratório do curso IMD0511.
>
> Autor: Adelson de Araujo (adelson.araujo@imd.ufrn.br).

<br>




### Parte 1 - Adquirir familiaridade com as operações básicas de desenvolvimento com GenAI.

Você irá explorar a biblioteca `google.generativeai` com o código demonstrado em sala de aula: [Tutorial de Introdução ao Desenvolvimento de IA Generativa com Python.ipynb](https://colab.research.google.com/drive/1fUFhU919QXAjCupxpfVA6Xsvs_wMxiiE?usp=sharing).

**Instruções**:

1. Configurar o ambiente.
2. Usar `generate_content()` para uma interação simples.
3. Usar `start_chat()` para criar uma conversa com histórico.
4. Inspecionar `chat.history` para entender como o histórico funciona.


In [73]:
import json
import os
from dotenv import load_dotenv

# Configure sua API Key
# IMPORTANTE: Nunca coloque sua chave diretamente no código em produção.
# Use variáveis de ambiente ou o gerenciador de "Secrets" do seu notebook (ex: Google Colab).
# Para este exemplo, vamos supor que você a colou aqui:
API_KEY = os.getenv("GEMINI_API_KEY")

if API_KEY is not None:
    print("API_KEY loaded successfully.")
else:
    print("API_KEY not found in environment variables.")

API_KEY loaded successfully.


In [74]:
import google.generativeai as genai

genai.configure(api_key=API_KEY)

# Escolher o modelo que vamos usar
llm = genai.GenerativeModel('models/gemini-2.5-flash')

print(llm.generate_content("ping").text)
print(llm.generate_content("já falei ping?").text)

Pong!
Não, até agora não falei "ping" nesta conversa.

Mas posso falar se quiser! Ou você está se referindo a algo específico onde "ping" seria relevante (como um comando de rede, um som, ou um aviso)?


In [75]:
chat = llm.start_chat()

resposta = chat.send_message("ping")
print(resposta.text)
resposta2 = chat.send_message("já falei ping?")
print(resposta2.text)

print("---Historico---")
print(chat.history)

Pong!
Sim, você disse "ping" lá no começo! E eu respondi "Pong!".
---Historico---
[parts {
  text: "ping"
}
role: "user"
, parts {
  text: "Pong!"
}
role: "model"
, parts {
  text: "já falei ping?"
}
role: "user"
, parts {
  text: "Sim, você disse \"ping\" lá no começo! E eu respondi \"Pong!\"."
}
role: "model"
]


### Parte 2 - Desenvolver o ciclo adaptativo de um ITS por meio de GenAI.


#### **Instruções**:



* Em [📄 Código-fonte do ITS], são fornecidas as funções já pré-implementadas e outras a serem implementadas.
* Implemente as funções `etapa_3_avaliacao_interacao` e `etapa_7_atualizar_aluno`, teste seu funcionamento.

* Rode o ITS na seção [🔁 Controle e execução do ITS]

* Refine os prompts utilizados nas outras funções para aprimorar/customizar seu ITS.

* Ao concluir, siga para a [📝 Tarefa].


#### 📄 **Código-fonte do ITS**

In [76]:
# --- Modelo de Domínio ---
def etapa_0_prep_modelo_dominio(llm, dominio, n_topicos=10, audiencia="4o ano do Ensino Fundamental"):
    """
    Define o Modelo de Domínio em tópicos, cada um com explicação, pré-requisito, e exercício.
    """
    prompt_dominio = f"""
    Você é especializado no domínio `{dominio}`.
    Pense nos {n_topicos} tópicos mais relevantes para a audiência "{audiencia}".
    Para cada tópico, precisará de (1) uma explicação espontâneamente simples,
    (2) a lista dos outros tópicos que são pré-requisitos deste tópico e (3) um
    exercício para avaliar a compreensão do tópico.
    As explicações e exercícios devem ser auto-contidos, não referenciando outras frases.
    O formato da sua saída deverá ser um json válido.
    """
    # Exemplo para servir de base para construção automática de novas bases de conhecimento
    exemplo_base_conhecimento = {
        "soma_fracoes": {
            "explicacao": "Para somar frações, primeiro encontre um denominador comum. Depois, some os numeradores.",
            "pre_requisitos": ["denominador_comum"],
            "exercicio": "Quanto é 1/2 + 1/4?"
        },
        "denominador_comum": {
            "explicacao": "O denominador comum é um múltiplo compartilhado pelos denominadores de duas ou mais frações.",
            "pre_requisitos":[],
            "exercicio": "Qual é o menor denominador comum para 1/3 e 1/5?"
        },
        "subtracao_fracoes": {
            "explicacao": "A subtração de frações segue a mesma lógica da soma: encontre um denominador comum e depois subtraia os numeradores.",
            "pre_requisitos": ["denominador_comum", "soma_fracoes"],
            "exercicio": "Quanto é 3/4 - 1/4?"
        }
    }
    # Transformar o dicionário acima em uma string mais legível:
    exemplo_base_conhecimento = json.dumps(exemplo_base_conhecimento, indent=4)
    prompt_dominio += f"\n Exemplo:\n {exemplo_base_conhecimento}"
    
    # Gerar modelo de domínio
    dominio_text = llm.generate_content(prompt_dominio).text
    dominio_clean = dominio_text.strip().replace("```json", "").replace("```", "")
    
    modelo_dominio = json.loads(dominio_clean)
    
    return modelo_dominio


In [77]:
# Testando

modelo_dominio = etapa_0_prep_modelo_dominio(llm, "fotosintese") # como chamar essa função corretamente?
print(modelo_dominio)

{'o_que_e_fotossintese': {'explicacao': 'Fotossíntese é o jeito que as plantas usam a luz do sol, a água e o ar para fazer o próprio alimento delas e crescerem.', 'pre_requisitos': [], 'exercicio': 'O que a fotossíntese ajuda a planta a fazer para ela mesma?'}, 'plantas_verdes_fabricas_de_comida': {'explicacao': 'As plantas verdes são como pequenas fábricas especiais que sabem produzir a sua própria comida. Isso é importante para elas terem energia e ficarem fortes.', 'pre_requisitos': ['o_que_e_fotossintese'], 'exercicio': "Por que dizemos que as plantas verdes são 'fábricas de comida'?"}, 'a_clorofila_magica_verde': {'explicacao': 'Clorofila é a substância que dá a cor verde às plantas. Ela é como um imã que ajuda a planta a pegar e usar a luz do sol para fazer o seu alimento.', 'pre_requisitos': ['o_que_e_fotossintese', 'plantas_verdes_fabricas_de_comida'], 'exercicio': 'Qual é a cor da clorofila e para que ela serve na planta?'}, 'a_luz_do_sol_energia_principal': {'explicacao': "A 

In [78]:
# --- Modelo do Aluno ---
def etapa_0_inicializar_aluno(modelo_dominio):
    """
    Inicializando o modelo do aluno como nível de maestria em cada um dos todos tópicos.
    """
    modelo_aluno = {topico: "iniciante" for topico in modelo_dominio}
    return modelo_aluno

In [79]:
# Testando

modelo_aluno = etapa_0_inicializar_aluno(modelo_dominio) # como chamar essa função corretamente?
print("---Modelo do Aluno---")
print(modelo_aluno)

---Modelo do Aluno---
{'o_que_e_fotossintese': 'iniciante', 'plantas_verdes_fabricas_de_comida': 'iniciante', 'a_clorofila_magica_verde': 'iniciante', 'a_luz_do_sol_energia_principal': 'iniciante', 'a_agua_planta_bebe': 'iniciante', 'o_gas_carbonico_planta_respira': 'iniciante', 'produzindo_acucar_comida_da_planta': 'iniciante', 'liberando_oxigenio_ar_bom_para_respirar': 'iniciante', 'as_folhas_pratos_solares': 'iniciante', 'importancia_da_fotossintese': 'iniciante'}


In [80]:
# --- Modelo Pedagógico ---
def etapa_1_selecao_proximo_topico(llm, modelo_aluno):
    """
    Usa o LLM para decidir qual a próxima tarefa (Macro-Adaptação).
    """
    
    topicos_disponiveis = list(modelo_aluno.keys())

    prompt_selecao_tarefa = f"""
    Você é um Sistema de Tutoria Inteligente.
    Decida o próximo tópico a ser ensinado com base no Modelo do Aluno a seguir:
    **Modelo do Aluno**:\n {json.dumps(modelo_aluno, indent=4)}\n

    **Tópicos Disponíveis (Escolha SOMENTE um destes)**:
    {topicos_disponiveis}
    
    Pense passo a passo para tomar sua decisão:
    1. Analise o histórico para entender o que o aluno já aprendeu.
    2. Com base na sua análise, identifique o próximo conceito mais lógico a ser ensinado, somente ensinando um tópico se os pré-requisitos forem julgados como aprendidos.
    3. Retorne sua decisão em um formato JSON com a chave "proximo_topico" (elemento presente nos Tópicos Disponíveis) e "raciocinio" (2-3 frases ultra-concisas).
    """
    content_text = llm.generate_content(prompt_selecao_tarefa).text
    content_clean = content_text.strip().replace("```json", "").replace("```", "")
    
    topico = json.loads(content_clean)

    return topico

In [81]:
# Testando

decisao = etapa_1_selecao_proximo_topico(llm, modelo_aluno) # como chamar essa função corretamente?
topico = decisao['proximo_topico']
print("---Próximo tópico---")
print(decisao)

---Próximo tópico---
{'proximo_topico': 'a_clorofila_magica_verde', 'raciocinio': "O aluno possui um entendimento inicial de todos os tópicos fundamentais da fotossíntese. Para aprofundar, o tópico 'a_clorofila_magica_verde' é o próximo passo lógico, pois explica o componente central que permite a captação da energia solar."}


In [82]:
def etapa_3_avaliacao_interacao_inicial(llm, chat, modelo_aluno):
    """
    Usa o LLM para avaliar a resposta inicial e atualizar o modelo do aluno.
    """
    mensagem_usuario = chat.history[-1].parts[0].text
    mensagem_agente = chat.history[-2].parts[0].text

    prompt_avaliacao = f"""
    Você é um avaliador em um Sistema de Tutoria Inteligente. Sua tarefa é avaliar a resposta de um aluno a um exercício e atualizar o nível de conhecimento dele sobre o tópico. Para isso considere o contexto:

    tutor: {mensagem_agente}
    aluno: {mensagem_usuario}

    Considere o modelo do aluno: {modelo_aluno}

    Instruções para Avaliação:
    1.  Identifique o Tópico: Com base no que o "Tutor disse", identifique qual dos "Tópicos Possíveis no Domínio" está sendo ensinado.
    2.  Avalie a Resposta: Analise a resposta do aluno.
    3.  Determine o Novo Nível: Com base na sua análise, determine o novo nível de conhecimento do aluno para o tópico identificado. Os níveis são: 'iniciante', 'intermediario', 'avancado'.
        - 'iniciante': Resposta incorreta ou vazia.
        - 'intermediario': Resposta parcialmente correta ou correta, mas superficial.
        - 'avancado': Resposta correta e que demonstra bom entendimento.
    4.  Forneça um Raciocínio: Justifique sua avaliação em uma frase.

    As explicações e exercícios devem ser auto-contidos, não referenciando outras frases.
    O formato da sua saída deverá ser um json válido.
    """

    exemplo = [
        {
        "topico_avaliado": "soma_fracoes",
        "raciocinio_avaliacao": "O aluno somou os numeradores e os denominadores diretamente, o que é um erro conceitual. Ele ainda não entendeu a necessidade de encontrar um denominador comum.",
        "novo_nivel": "iniciante"
        },
        {
        "topico_avaliado": "soma_fracoes",
        "raciocinio_avaliacao": "O aluno identificou corretamente o denominador comum e converteu uma das frações, mas não finalizou a soma. Ele compreende o processo, mas precisa de um pequeno empurrão para completar a tarefa.",
        "novo_nivel": "intermediario"
        },
        {
        "topico_avaliado": "denominador_comum",
        "raciocinio_avaliacao": "O aluno forneceu a resposta correta para o menor denominador comum, demonstrando que compreendeu o conceito fundamental para operações com frações.",
        "novo_nivel": "avancado"
        }
    ]

    exemplo = json.dumps(exemplo, indent=4)
    prompt_avaliacao += f"\n Exemplo:\n {exemplo}"

        # Gerar modelo de domínio
    avaliacao = llm.generate_content(prompt_avaliacao).text
    dominio_clean = avaliacao.strip().replace("```json", "").replace("```", "")
    
    avaliacao_json = json.loads(dominio_clean)

    return avaliacao_json, modelo_aluno

In [83]:
# Testando

chat = llm.start_chat()

mensagem_aluno = "ping"

resposta = chat.send_message(mensagem_aluno)

avaliacao, modelo_aluno = etapa_3_avaliacao_interacao_inicial(llm, chat, modelo_aluno)
print("---Modelo do Aluno (pós avaliação de interação inicial) ---")
print(avaliacao)

---Modelo do Aluno (pós avaliação de interação inicial) ---
[{'topico_avaliado': 'o_que_e_fotossintese', 'raciocinio_avaliacao': "A resposta do aluno foi um simples reconhecimento ('pong') à mensagem do tutor ('ping'), não demonstrando nenhum conhecimento ou engajamento com um tópico específico do domínio. Portanto, não houve avanço no entendimento do conceito de fotossíntese.", 'novo_nivel': 'iniciante'}]


In [84]:
def etapa_45_decidir_e_gerar_feedback(chat, exercicio):
    """
    Usa o LLM para avaliar a resposta e gerar feedback adaptativo (Micro-Adaptação).
    """
    resposta_aluno = chat.history[-1].parts[0].text
    prompt_feedback = f"""
    --- Decisão de feedback ---
    Como ITS, Avalie a resposta do aluno e forneça um feedback útil.
    Exercício Proposto: *"{exercicio}"*
    Resposta do Aluno: **"{resposta_aluno}"**
    Pense passo a passo:
    1. A resposta está correta?.
    2. Se incorreta, identifique o equívoco.
    3. Gere uma dica sem dar a resposta.
    4. Formule sua resposta final para o aluno.
    ----------------------------
    Feedback:
    """
    resposta = chat.send_message(prompt_feedback)
    return resposta.text

In [85]:
# Testando

#corrige topico invalido
topico_valido = list(modelo_dominio.keys())[0]

feedback = etapa_45_decidir_e_gerar_feedback(chat, exercicio=modelo_dominio[topico_valido]['exercicio'])
print(feedback)

Olá! Parece que ainda estamos na nossa brincadeira de "ping" e "pong" de antes! 😊

A sua resposta "pong" não corresponde ao que perguntamos sobre a fotossíntese. Essa pergunta é sobre um processo biológico muito importante para as plantas!

**Dica:** Pensa no que a planta precisa para sobreviver e crescer. A fotossíntese é o processo que ela usa para *fabricar* o seu próprio alimento ou a sua própria energia. O que ela produz para se manter viva e forte?


In [86]:
# --- Modelo do Aluno ---
def etapa_7_atualizacao_pos_feedback(chat, modelo_aluno):
    """
    Atualiza o modelo do aluno com base no último ciclo de conversa.
    """

    if len(chat.history) > 3:
      # Atualize o modelo do aluno com  base n as últimas três mensagens do
      # historico: interacao inicial, feedback, e interacao final do ciclo.
        ciclo_interacao = chat.history[-3:]
        
        # 1. Chamar a função de avaliação existente (etapa_3) pra atualizar pos-feedback
        avaliacao_json, _ = etapa_3_avaliacao_interacao_inicial(llm, chat, modelo_aluno)
        
        # então pegamos o primeiro item.
        if avaliacao_json:
            avaliacao = avaliacao_json[0]
            topico_avaliado = avaliacao.get("topico_avaliado")
            novo_nivel = avaliacao.get("novo_nivel")

            if topico_avaliado and novo_nivel and topico_avaliado in modelo_aluno:
                modelo_aluno[topico_avaliado] = novo_nivel
            
    return modelo_aluno

In [87]:
# Testando

modelo_aluno = etapa_7_atualizacao_pos_feedback(chat, modelo_aluno)
print(modelo_aluno)

{'o_que_e_fotossintese': 'iniciante', 'plantas_verdes_fabricas_de_comida': 'iniciante', 'a_clorofila_magica_verde': 'iniciante', 'a_luz_do_sol_energia_principal': 'iniciante', 'a_agua_planta_bebe': 'iniciante', 'o_gas_carbonico_planta_respira': 'iniciante', 'produzindo_acucar_comida_da_planta': 'iniciante', 'liberando_oxigenio_ar_bom_para_respirar': 'iniciante', 'as_folhas_pratos_solares': 'iniciante', 'importancia_da_fotossintese': 'iniciante'}


#### **🔁 Controle e execução do ITS**

In [88]:
def sistema_tutoria_inteligente_genai(llm, dominio, n_topicos=10, audiencia="4o ano do Ensino Fundamental"):
    """
    Inicia uma nova sessão de tutoria.
    """
    prompt_sistema = {"role": "model", "parts": ["Você é um Sistema de Tutoria Inteligente (ITS)."]}
    chat = llm.start_chat(history=[prompt_sistema])

    modelo_dominio = etapa_0_prep_modelo_dominio(llm, dominio, n_topicos=10, audiencia="4o ano do Ensino Fundamental")
    modelo_aluno = etapa_0_inicializar_aluno(modelo_dominio)

    # Enquanto houver topicos a aprender...
    while modelo_aluno:

        # Etapa 1
        decisao = etapa_1_selecao_proximo_topico(llm, modelo_aluno)

        explicacao = modelo_dominio[decisao['proximo_topico']]['explicacao']
        exercicio = modelo_dominio[decisao['proximo_topico']]['exercicio']

        print(explicacao)
        print(exercicio)

        # Etapa 2
        mensagem_usuario = input("Usuário:")
        chat.send_message(mensagem_usuario)

        # Etapa 3
        avaliacao_inicial, modelo_aluno_temp = etapa_3_avaliacao_interacao_inicial(llm, chat, modelo_aluno)

        # Etapas 4 e 5
        feedback = etapa_45_decidir_e_gerar_feedback(chat, exercicio)

        print(feedback)

        # Etapa 6
        mensagem_usuario = input("Usuário:") # etapa_7
        chat.send_message(mensagem_usuario)

        # Etapa 7
        modelo_aluno_feedback = etapa_7_atualizacao_pos_feedback(chat, modelo_aluno_temp)

        # Condição de parada: ter aprendido todos os tópicos
        if modelo_aluno_feedback[decisao['proximo_topico']] == 'avançado':
        # Removendo tópico já aprendido
            modelo_aluno_feedback.pop(decisao['proximo_topico'])

    return chat

In [89]:

chat_session = sistema_tutoria_inteligente_genai(llm, "fotosintese")

A fotossíntese é um processo especial que as plantas usam para fazer seu próprio alimento, usando a luz do sol, água e o ar (gás carbônico).
Qual é o nome do processo que as plantas usam para fazer sua própria comida?
Sua resposta final para o aluno:

Olá! Sua descrição do processo é **excelente e muito completa**! Você explicou de forma muito clara os componentes (luz, CO2, água), os produtos (glicose, oxigênio) e até onde acontece (cloroplastos, clorofila).

No entanto, a pergunta era sobre o **nome específico** desse processo que as plantas usam para fazer sua própria comida. Você consegue se lembrar qual é o termo que usamos para designar todo esse fenômeno?

Pense na palavra-chave que usamos na nossa conversa inicial sobre como as plantas adquirem energia usando luz.
Durante a fotossíntese, as plantas fazem um tipo de açúcar que é como a energia que elas precisam para crescer e ficar fortes.
O que a planta produz para ela mesma durante a fotossíntese? ( ) Água ( ) Sol ( ) Alimento

KeyboardInterrupt: Interrupted by user

### Parte 3 - 📝 Tarefa

Submeter no SIGAA o link deste colab notebook prenchido, com última alteração feita antes da Data de fechamento da tarefa.

Seu notebook deve conter um relatório formulado abaixo, respondendo à seguintes perguntas:

<br>

-----------



> Nome: José Manoel Freitas da Silva

##### 1. Avalie a eficácia do seu ITS para apoiar a aprendizagem de três tipos de alunos: o aluno iniciante, o intermediário e o avançado.

Com base nos testes, vemos que o ITS pode ser eficaz para alunos iniciantes e intermediários, uma vez que as perguntas feitas com base no modelo de domínio são simples e o feedback traz um reforço positivo. Já para alunos avançados, as perguntas simples podem levar a um baixo engajamento.

##### 2. Faça uma simulação de tutoria fictícia de um **aluno iniciante** (interpretado por você) nos domínios de `fotossíntese` e `coesão textual`, quais são os prós e os contras dessa abordagem de tutoria?

Como pontos fortes, o ITS fornece um conteúdo bastante acessível com feedback imediato, crucial para corrigir possíveis equívocos no aprendizado, usando do reforço positivo.

Por outro lado, existe um risco de inconsistência e de gerar respostas fora do esperado, resultando em uma conversa que diverge do tema principal.

##### 3. Quais seriam bons próximos passos para aprimorar essa implementação de tutoria?

Para os próximos passos, deve-se aprimorar os prompts para as redes neurais e adaptar o código para suportar respostas falhas e fora da formatação esperada.

.